In [5]:
# 17.2

import sqlite3
import os
from tabulate import tabulate    

def fetch_and_display_table_data(db_path, table_name, output_file):
    with sqlite3.connect(db_path) as conn:
        cursor = conn.cursor()

        # Execute the query to fetch all data from the specified table
        cursor.execute(f"SELECT * FROM {table_name}")

        # Fetch all rows
        rows = cursor.fetchall()

        # Extract column headers from the cursor description
        columns = [description[0] for description in cursor.description]

        # Using tabulate to format the data in a table-like structure
        formatted_table = tabulate(rows, headers=columns, tablefmt='grid')

        # Write the formatted table data to the output file
        with open(output_file, 'w') as file:
            file.write(formatted_table)

# Define the path to the database
db_path = os.path.join('ch17', 'books.db')

# Output file path
output_file = 'books_output.txt'

# Fetching and writing data from the 'titles' table to the output file
fetch_and_display_table_data(db_path, 'titles', output_file)

print("Data written to", output_file)

# Display data from a table (replace 'your_table_name' with an actual table name)



Data written to books_output.txt


In [7]:
# 17.1

import sqlite3
import pandas as pd

def execute_sql_script(db_path, sql_file_path):
    with sqlite3.connect(db_path) as conn, open(sql_file_path, 'r') as file:
        script = file.read()
        conn.executescript(script)

# Function to perform a query and save the result to a CSV file
def query_to_csv(db_path, query, csv_path, params=None):
    with sqlite3.connect(db_path) as conn:
        df = pd.read_sql_query(query, conn, params=params)
    df.to_csv(csv_path, index=False)

# Paths to the SQL file and the new database
sql_file_path = r"C:\Users\ryanc\OneDrive\Documents\COD Courses\Advanced_Python\Unit12\ch17\books.sql"  
db_path = r"C:\Users\ryanc\OneDrive\Documents\COD Courses\Advanced_Python\Unit12\books.db"         

# Execute the SQL script to create the database
execute_sql_script(db_path, sql_file_path)

# Query to select all authors' last names in descending order and save to CSV
query_authors = "SELECT last FROM authors ORDER BY last DESC"
csv_authors = r"C:\Users\ryanc\OneDrive\Documents\COD Courses\Advanced_Python\Unit12\names.csv"   
query_to_csv(db_path, query_authors, csv_authors)

# Query to select all book titles in ascending order and save to CSV
query_titles = "SELECT title FROM titles ORDER BY title ASC"
csv_titles = r"C:\Users\ryanc\OneDrive\Documents\COD Courses\Advanced_Python\Unit12\titles.csv"   
query_to_csv(db_path, query_titles, csv_titles)

# Query to select all books for a specific author and save to CSV

author_last_name = 'Deitel'  
query_books_for_author = """
SELECT t.title, t.copyright, t.isbn
FROM titles t
JOIN author_ISBN ai ON t.isbn = ai.isbn
JOIN authors a ON ai.id = a.id
WHERE a.last = ?
ORDER BY t.title ASC
"""
csv_books_for_author = r"C:\Users\ryanc\OneDrive\Documents\COD Courses\Advanced_Python\Unit12\authors.csv"   
query_to_csv(db_path, query_books_for_author, csv_books_for_author, params=[author_last_name])

# Insert a new author into the authors table
new_author = ("Jane", "Doe")  
with sqlite3.connect(db_path) as conn:
    conn.execute("INSERT INTO authors (first, last) VALUES (?, ?)", new_author)

# Insert a new title for the new author
new_title = ("1234567890", "New Book Title", 1, "2023")
new_author_id = conn.execute("SELECT id FROM authors WHERE first = ? AND last = ?", new_author).fetchone()[0]
with sqlite3.connect(db_path) as conn:
    conn.execute("INSERT INTO titles (isbn, title, edition, copyright) VALUES (?, ?, ?, ?)", new_title)
    conn.execute("INSERT INTO author_ISBN (id, isbn) VALUES (?, ?)", (new_author_id, new_title[0]))